# BMG Labtech CLARIOstar (Plus)

| Summary | Photo |
|---------|-------|
| <ul style="font-size:15px; line-height:1.6; margin-top:0;"> <li><a href="https://www.bmglabtech.com/en/clariostar-plus/" target="_blank"><b>OEM Link</b></a></li> <li><b>Communication Protocol / Hardware:</b> Serial (FTDI) / USB-A</li> <li><b>Communication Level:</b> Firmware</li> <li><b>Measurement Modes:</b> Absorbance, Luminescence, Fluorescence</li> <li><b>Plate Delivery:</b> Loading tray</li> <li><b>Additional Features:</b> Temperature control, Shaking, Configurable scan modes</li> <li>VID:PID <code>0403:BB68</code></li> </ul> | <div style="width:320px; text-align:center;"> ![clariostar](img/bmg-labtech-clariostar-plus.png) <br><i>Figure: BMG Labtech CLARIOstar Plus</i> </div> |

---
## Setup Instructions (Physical)

The CLARIOstar and CLARIOstar Plus require a minimum of two cable connections to be operational:
1. Power cord (standard IEC C13)
2. USB cable (USB-B with security screws at CLARIOstar end; USB-A at control PC end)

Optional:
If you have a plate stacking unit to use with the CLARIOstar (Plus), an additional RS-232 port is available on the CLARIOstar (Plus).

---
## Setup Instructions (Programmatic)

To control the BMG Labtech CLARIOstar (Plus), create a `PlateReader` frontend instance that uses a `CLARIOstarBackend` as its backend.

For convenience, store the backend as a separate variable so you can access CLARIOstar-specific features directly.

Set `protocol_mode` below to `"execution"` when connected to real hardware, or `"simulation"` to run the notebook end-to-end without a device.

In [1]:
protocol_mode = "simulation" # "execution" or "simulation"

In [2]:
from pylabrobot.plate_reading import PlateReader

if protocol_mode == "execution":
    from pylabrobot.plate_reading.bmg_labtech import CLARIOstarBackend
    clariostar_backend = CLARIOstarBackend()

elif protocol_mode == "simulation":
    from pylabrobot.plate_reading.bmg_labtech import CLARIOstarSimulatorBackend
    clariostar_backend = CLARIOstarSimulatorBackend()

pr = PlateReader(
    name="CLARIOstar",
    backend=clariostar_backend,
    size_x=0.0,
    size_y=0.0,
    size_z=0.0
)

If you have multiple FTDI devices connected, pass a `device_id` to the backend to select the correct one:

```python
clariostar_backend = CLARIOstarBackend(device_id="FT1234AB")
```

In [3]:
await pr.setup()

```{note}
Expected behaviour: the machine performs its initialization routine (FTDI connection, baudrate configuration, firmware initialization, and EEPROM data request).
```

---
## Defining a Plate

Every measurement requires a plate to be assigned to the plate reader. The backend dynamically encodes the plate geometry (dimensions, well positions, and well count) into the binary protocol — so you can use **any** PLR plate definition.

Here we use a standard Corning 96-well flat-bottom plate as an example:

In [4]:
from pylabrobot.resources.corning import Cor_96_wellplate_360ul_Fb

plate = Cor_96_wellplate_360ul_Fb("test_plate")
pr.assign_child_resource(plate)

---
## Usage: Loading Tray

The CLARIOstar loads and unloads plates via a motorized loading tray.

In [5]:
# Open the loading tray (plate out)
await pr.open()

In [6]:
# Place your plate on the tray (manually or via a robotic arm), then close it (plate in)
await pr.close()

---
## Usage: Querying Machine Status

The CLARIOstar reports its state as a set of bitwise status flags. You can query them at any time to check what the machine is doing.

The available flags are:

| Flag | Meaning |
|------|---------|
| `STANDBY` | Machine is in standby mode |
| `VALID` | Status response is valid |
| `BUSY` | Machine is currently executing a command |
| `RUNNING` | A measurement run is in progress |
| `UNREAD_DATA` | Measurement data is available to read |
| `INITIALIZED` | Firmware has been initialized |
| `LID_OPEN` | The machine lid is open |
| `OPEN` | The loading tray is open |
| `PLATE_DETECTED` | A plate is detected on the tray |
| `Z_PROBED` | Z-height probing has completed |
| `ACTIVE` | Machine is active |
| `FILTER_COVER_OPEN` | The filter cover is open |

In [7]:
from pylabrobot.plate_reading.bmg_labtech.clario_star_backend import StatusFlag

status = await clariostar_backend.get_status()
print(status)

{<StatusFlag.INITIALIZED: 'INITIALIZED'>, <StatusFlag.VALID: 'VALID'>, <StatusFlag.PLATE_DETECTED: 'PLATE_DETECTED'>}


In [8]:
# Check individual flags
if StatusFlag.PLATE_DETECTED in status:
    print("Plate is on the tray")
else:
    print("No plate detected")

if StatusFlag.BUSY in status:
    print("Machine is busy")
else:
    print("Machine is ready")

Plate is on the tray
Machine is ready


---
## Usage: Measuring Luminescence

Luminescence measures light emitted by a chemical or biological reaction (no excitation light source).

### Parameters

| Parameter | Type | Default | Description |
|-----------|------|---------|-------------|
| `focal_height` | `float` | `13` | Focal height in mm (0–25) |

### Basic luminescence read (full plate)

In [9]:
results = await pr.read_luminescence(
    focal_height=13.0,
    use_new_return_type=True
)

# results is a list of dicts, each containing:
#   "data": 8x12 grid of float values (rows x columns)
#   "temperature": float (Celsius)
#   "time": float (unix timestamp)
print(f"Temperature: {results[0]['temperature']} \u00b0C")
print(f"Well A1 value: {results[0]['data'][0][0]}")

Temperature: 26.0 °C
Well A1 value: 3891.5845732141615


---
## Usage: Measuring Absorbance

Absorbance measures how much light a sample absorbs at a given wavelength.
Results can be reported as optical density (OD) or percent transmittance.

### Parameters

| Parameter | Type | Default | Description |
|-----------|------|---------|-------------|
| `wavelength` | `int` | *required* | Wavelength in nm (200–1000) |
| `report` | `"OD"` or `"transmittance"` | `"OD"` | Output format |
| `wavelengths` | `List[int]` | — | Multiple wavelengths (1–8), overrides `wavelength` |
| `flashes` | `int` | `22` | Number of flashes per well (0–200) |
| `settling_time` | `int` | `0` | Settling time in deciseconds (0–10) |

### Single-wavelength absorbance (OD)

In [10]:
results = await pr.read_absorbance(
    wavelength=600,
    use_new_return_type=True
)

# results is a list with one dict per wavelength, each containing:
#   "wavelength": int (nm)
#   "data": 8x12 grid of OD values
#   "temperature": float (Celsius)
#   "time": float (unix timestamp)
print(f"OD600 of well A1: {results[0]['data'][0][0]}")
print(f"Temperature: {results[0]['temperature']} \u00b0C")

OD600 of well A1: 0.5615490321313947
Temperature: 26.0 °C


### Absorbance as transmittance

In [ ]:
results = await pr.read_absorbance(
    wavelength=450,
    report="transmittance",
    use_new_return_type=True
)

# Transmittance is in percent (0-100)
print(f"Transmittance at 450nm, well A1: {results[0]['data'][0][0]}%")

### Multi-wavelength absorbance

Read up to 8 wavelengths in a single run. This is faster than running separate measurements because the plate is only scanned once.

Pass `wavelengths` as a backend keyword argument (this overrides the `wavelength` parameter):

In [ ]:
results = await pr.read_absorbance(
    wavelength=600,             # ignored when wavelengths is provided
    wavelengths=[260, 280, 450, 600, 750],
    use_new_return_type=True
)

# One result dict per wavelength
for r in results:
    print(f"Wavelength {r['wavelength']} nm -> A1 OD: {r['data'][0][0]}")

---
## Usage: Measuring Fluorescence

Fluorescence measures light emitted by a fluorophore after excitation at a specific wavelength. The CLARIOstar uses monochromators for both excitation and emission wavelength selection.

### Parameters

| Parameter | Type | Default | Description |
|-----------|------|---------|-------------|
| `excitation_wavelength` | `int` | *required* | Excitation center wavelength in nm |
| `emission_wavelength` | `int` | *required* | Emission center wavelength in nm |
| `focal_height` | `float` | *required* | Focal height in mm (0–25) |
| `gain` | `int` | `1000` | Detector gain |
| `ex_bandwidth` | `int` | `20` | Excitation bandwidth in nm |
| `em_bandwidth` | `int` | `40` | Emission bandwidth in nm |
| `dichroic` | `int` | auto | Dichroic wavelength \u00d7 10 (auto-calculated as `(ex + em) * 5`) |
| `flashes` | `int` | `100` | Number of flashes per well (0–200) |
| `settling_time` | `int` | `0` | Settling time in deciseconds (0–10) |
| `bottom_optic` | `bool` | `False` | Read from bottom instead of top |

### Basic fluorescence read (e.g. GFP: ex 485 / em 528)

In [ ]:
results = await pr.read_fluorescence(
    excitation_wavelength=485,
    emission_wavelength=528,
    focal_height=8.5,
    use_new_return_type=True
)

# results is a list with one dict containing:
#   "ex_wavelength": int, "em_wavelength": int,
#   "data": 8x12 grid, "temperature": float, "time": float
print(f"GFP signal at A1: {results[0]['data'][0][0]}")
print(f"Temperature: {results[0]['temperature']} \u00b0C")

### Fluorescence with custom gain, bandwidth, and flashes

In [ ]:
results = await pr.read_fluorescence(
    excitation_wavelength=485,
    emission_wavelength=528,
    focal_height=8.5,
    gain=1500,           # increase gain for weak signals
    ex_bandwidth=10,     # narrow excitation bandwidth for better specificity
    em_bandwidth=20,     # narrow emission bandwidth
    flashes=150,         # more flashes for better signal-to-noise
    use_new_return_type=True
)

### Bottom-optic fluorescence

Use bottom-optic reading when working with cell monolayers, adherent cells, or clear-bottom plates where top-reading would be affected by the meniscus or lid:

In [ ]:
results = await pr.read_fluorescence(
    excitation_wavelength=544,
    emission_wavelength=590,
    focal_height=4.5,
    bottom_optic=True,
    use_new_return_type=True
)

---
## Usage: Partial Well Selection

All three measurement modes (luminescence, absorbance, fluorescence) support reading a subset of wells instead of the full plate. This can significantly speed up reads when you only need data from specific wells.

Unread wells are filled with `None` in the output grid.

In [ ]:
# Read only column 1 (wells A1-H1)
column_1_wells = [plate.get_well(f"{row}1") for row in "ABCDEFGH"]

results = await pr.read_absorbance(
    wavelength=600,
    wells=column_1_wells,
    use_new_return_type=True
)

# Only column 1 has values; all other wells are None
for row_idx, row_letter in enumerate("ABCDEFGH"):
    val = results[0]['data'][row_idx][0]  # column 0 = column 1
    print(f"Well {row_letter}1: OD = {val}")

In [ ]:
# Read specific wells by name
selected_wells = [plate.get_well("A1"), plate.get_well("D6"), plate.get_well("H12")]

results = await pr.read_luminescence(
    focal_height=13.0,
    wells=selected_wells,
    use_new_return_type=True
)

In [ ]:
# Partial well selection works for fluorescence too
row_A_wells = [plate.get_well(f"A{col}") for col in range(1, 13)]

results = await pr.read_fluorescence(
    excitation_wavelength=485,
    emission_wavelength=528,
    focal_height=8.5,
    wells=row_A_wells,
    use_new_return_type=True
)

---
## Usage: Scan Mode Configuration

The scan mode controls how the plate reader traverses the plate during a measurement. Configurable via backend keyword arguments on all three read methods.

### Parameters

| Parameter | Type | Default | Description |
|-----------|------|---------|-------------|
| `start_corner` | `StartCorner` | `TOP_LEFT` | Which corner to begin reading from |
| `unidirectional` | `bool` | `False` (lum/fl), `True` (abs) | Read in one direction only (vs. bidirectional zigzag) |
| `vertical` | `bool` | `False` | Read columns instead of rows |
| `flying_mode` | `bool` | `False` | Continuous movement (fluorescence only, max 3 flashes) |

Available start corners:
- `StartCorner.TOP_LEFT` (default)
- `StartCorner.TOP_RIGHT`
- `StartCorner.BOTTOM_LEFT`
- `StartCorner.BOTTOM_RIGHT`

In [ ]:
from pylabrobot.plate_reading.bmg_labtech.clario_star_backend import StartCorner

In [ ]:
# Read from bottom-right corner, vertical scan, unidirectional
results = await pr.read_absorbance(
    wavelength=600,
    start_corner=StartCorner.BOTTOM_RIGHT,
    unidirectional=True,
    vertical=True,
    use_new_return_type=True
)

In [ ]:
# Flying mode for fast fluorescence reads (max 3 flashes)
results = await pr.read_fluorescence(
    excitation_wavelength=485,
    emission_wavelength=528,
    focal_height=8.5,
    flying_mode=True,
    flashes=3,
    start_corner=StartCorner.TOP_LEFT,
    use_new_return_type=True
)

```{note}
Flying mode keeps the optics head moving continuously instead of stopping at each well. This is much faster but limits you to a maximum of 3 flashes per well. Only available for fluorescence.
```

---
## Usage: Pre-Measurement Shaking

The CLARIOstar has a built-in shaker. When configured as part of a measurement call, the plate is shaken **immediately before** the optics head begins reading. This is useful for resuspending cells or mixing reagents right before a read.

```{note}
Currently, shaking is only available as a step embedded in a measurement run. Standalone shaking (shake without measuring) is a planned feature — see [Planned Features](#planned-features) below.
```

### Parameters

| Parameter | Type | Default | Description |
|-----------|------|---------|-------------|
| `shake_type` | `ShakerType` | `ORBITAL` | Type of shaking motion |
| `shake_speed_rpm` | `int` | `0` | Speed in RPM (100–700, in steps of 100) |
| `shake_duration_s` | `int` | `0` | Duration in seconds (0 = no shaking) |

Available shaker types:

| Type | Description | Max RPM |
|------|-------------|--------:|
| `ShakerType.ORBITAL` | Circular orbit | 700 |
| `ShakerType.LINEAR` | Back and forth in a line | 700 |
| `ShakerType.DOUBLE_ORBITAL` | Figure-eight pattern | 700 |
| `ShakerType.MEANDER` | Meandering path | **300** |

In [ ]:
from pylabrobot.plate_reading.bmg_labtech.clario_star_backend import ShakerType

In [ ]:
# Orbital shake at 300 RPM for 5 seconds before reading luminescence
results = await pr.read_luminescence(
    focal_height=13.0,
    shake_type=ShakerType.ORBITAL,
    shake_speed_rpm=300,
    shake_duration_s=5,
    use_new_return_type=True
)

In [ ]:
# Linear shake at 500 RPM for 10 seconds before reading absorbance
results = await pr.read_absorbance(
    wavelength=600,
    shake_type=ShakerType.LINEAR,
    shake_speed_rpm=500,
    shake_duration_s=10,
    use_new_return_type=True
)

In [ ]:
# Double orbital shake before fluorescence
results = await pr.read_fluorescence(
    excitation_wavelength=485,
    emission_wavelength=528,
    focal_height=8.5,
    shake_type=ShakerType.DOUBLE_ORBITAL,
    shake_speed_rpm=400,
    shake_duration_s=3,
    use_new_return_type=True
)

```{warning}
The `MEANDER` shaker type is limited to a maximum of 300 RPM. Exceeding this will raise a `ValueError`.
```

---
## Usage: Combining Features

All features can be combined in a single read call. Here is an example that uses partial well selection, pre-measurement shaking, and a custom scan configuration together:

In [ ]:
# Read fluorescence from wells A1-A12 with orbital shake, bottom-right start, and custom gain
row_A = [plate.get_well(f"A{col}") for col in range(1, 13)]

results = await pr.read_fluorescence(
    excitation_wavelength=544,
    emission_wavelength=590,
    focal_height=6.0,
    wells=row_A,
    gain=2000,
    flashes=50,
    bottom_optic=True,
    shake_type=ShakerType.ORBITAL,
    shake_speed_rpm=300,
    shake_duration_s=5,
    start_corner=StartCorner.BOTTOM_RIGHT,
    unidirectional=True,
    use_new_return_type=True
)

# Print row A results
for col_idx in range(12):
    val = results[0]['data'][0][col_idx]
    print(f"A{col_idx+1}: {val}")

---
## Closing Connection

---
(planned-features)=
## Planned Features

The CLARIOstar firmware supports several additional features that are documented in the BMG ActiveX/DDE manual but **not yet implemented** in the PyLabRobot backend. These require USB traffic captures of the corresponding firmware byte sequences.

### Standalone Shaking

The firmware supports a dedicated `Shake` command independent of any measurement run. This would enable:

```python
# Planned API (not yet implemented)
await clariostar_backend.shake(
    shake_type=ShakerType.ORBITAL,
    speed_rpm=300,
    duration_s=60
)
```

Per the OEM manual, standalone shaking supports:
- 5 shake types: orbital, linear, double orbital, meander corner well, orbital corner well
- Speed: 100–700 RPM (up to 1100 with high-speed shaking option)
- Duration: 1–3600 seconds
- Optional X/Y position parameters

### Temperature Control

The firmware supports a `Temp` command to control the built-in incubator:

```python
# Planned API (not yet implemented)
await clariostar_backend.set_temperature(37.0)   # heat to 37 °C
temp = await clariostar_backend.get_temperature() # read current temp
await clariostar_backend.set_temperature(0.0)     # switch off incubator
```

Per the OEM manual:
- Target temperature: 25.0–45.0 °C (in 0.1 °C increments), extended range 10–65 °C with optional upgrade
- Two temperature sensors: bottom heating plate (`Temp1`) and top heating plate (`Temp2`)
- `0.0` = switch off incubator, `0.1` = monitoring only (no heating)

### Idle Movement

The firmware supports an `IdleMove` command for continuous plate movement between measurements (useful during long incubation periods):

```python
# Planned API (not yet implemented)
await clariostar_backend.idle_move(mode="orbital", speed_rpm=200, duration_s=300)
await clariostar_backend.idle_move(mode="cancel")  # stop idle movement
```

Per the OEM manual, 7 modes are available including linear corner-to-corner movement, incubation position waiting, and various shaking patterns with configurable on/off cycling.

### Injector System

The CLARIOstar supports up to two injector pumps (`Pump1` / `Pump2`) for dispensing reagents during kinetic reads. Not yet implemented.

```{note}
To help implement any of these features, capture USB traffic while executing the corresponding command in the BMG MARS software and open an issue on the [PyLabRobot GitHub](https://github.com/PyLabRobot/pylabrobot/issues).
```

In [ ]:
pr.unassign_child_resource(plate)

In [ ]:
await pr.stop()

This closes the FTDI connection. After calling `stop()`, you must call `setup()` again before using the plate reader.